In [1]:
import numpy as np 
import pandas as pd
from sklearn import linear_model

In [2]:
DATA_PATH = '~/google_drive/data/quora/'
TRAIN_FILE = '{}{}'.format(DATA_PATH,'train.csv')
TEST_FILE = '{}{}'.format(DATA_PATH,'test.csv') 

In [3]:
train = pd.read_csv(TRAIN_FILE)
test = pd.read_csv(TRAIN_FILE)

In [ ]:
train.head()

In [4]:
test.head()

,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [5]:
train_data = [ [vec[0],vec[1].split(' '),vec[2]] for vec in test.as_matrix()]                                                 
train_data[:20]

[['00002165364db923c7e6',
  ['How',
   'did',
   'Quebec',
   'nationalists',
   'see',
   'their',
   'province',
   'as',
   'a',
   'nation',
   'in',
   'the',
   '1960s?'],
  0],
 ['000032939017120e6e44',
  ['Do',
   'you',
   'have',
   'an',
   'adopted',
   'dog,',
   'how',
   'would',
   'you',
   'encourage',
   'people',
   'to',
   'adopt',
   'and',
   'not',
   'shop?'],
  0],
 ['0000412ca6e4628ce2cf',
  ['Why',
   'does',
   'velocity',
   'affect',
   'time?',
   'Does',
   'velocity',
   'affect',
   'space',
   'geometry?'],
  0],
 ['000042bf85aa498cd78e',
  ['How',
   'did',
   'Otto',
   'von',
   'Guericke',
   'used',
   'the',
   'Magdeburg',
   'hemispheres?'],
  0],
 ['0000455dfa3e01eae3af',
  ['Can',
   'I',
   'convert',
   'montra',
   'helicon',
   'D',
   'to',
   'a',
   'mountain',
   'bike',
   'by',
   'just',
   'changing',
   'the',
   'tyres?'],
  0],
 ['00004f9a462a357c33be',
  ['Is',
   'Gaza',
   'slowly',
   'becoming',
   'Auschwitz,',
   'Dac